In [1]:
import numpy as np
from rubin_sim.satellite_constellations import Constellation, starlink_tles_v1, starlink_tles_v2
import pandas as pd
from rubin_sim.utils import point_to_line_distance

In [2]:
# need to add a new method on the constallation to do the checking the way we want
class SConstellation(Constellation):
    def check_positions(self, positions_ra, positions_dec, mjds, visit_times, mask_width=1./60.):
        """Check if RA,dec,mjd spot is hit by a satellite streak
        
        Parameters
        ----------
        positions_ra : np.array
            The RA of each solar system detection (degrees).
        positions_dec : np.array
            The Dec of each solar system detection (degrees).
        mjds : np.array
            The MJD of each detection
        visit_times : np.array
            The total visit times for each detection. Typically exposure time plus 
            any additional read time or shutter motion time. (seconds)
        mask_width : float
            The width of the expected streak (degrees)
        
        Returns
        -------
        index values for positions that were hit by a streak
        """
        # convert all input to radians
        positions_ra = np.radians(positions_ra)
        positions_dec = np.radians(positions_dec)
        mask_width = np.radians(mask_width)
        
        visit_times = visit_times / 3600.0 / 24.0  # convert seconds to days
        
        input_id_indx_oned = np.arange(positions_ra.size, dtype=int)
        
        sat_ra_1, sat_dec_1, sat_alt_1, sat_illum_1 = self.paths_array(mjds)
        mjd_end = mjds + visit_times
        sat_ra_2, sat_dec_2, sat_alt_2, sat_illum_2 = self.paths_array(mjd_end)
    
        
        # broadcast the object positions to be the same shape as the satellite arrays.
        pointing_ras_rad = np.broadcast_to(positions_ra, sat_ra_1.shape)
        pointing_decs_rad = np.broadcast_to(positions_dec, sat_ra_1.shape)
        input_id_indx = np.broadcast_to(input_id_indx_oned, sat_ra_1.shape)
        
        above_illum_indx = np.where(
            ((sat_alt_1 > self.alt_limit_rad) | (sat_alt_2 > self.alt_limit_rad))
            & ((sat_illum_1 == True) | (sat_illum_2 == True))
        )

        # XXXX--this is assuming satellites travel on straight lines on the sphere
        # I'm not sure that's a super great assumption, but maybe works out
        # statistically where we get some hits right and some wrong
        
        # point_to_line_distance can take arrays, but they all need to be the same shape,
        # thus why we broadcast pointing ra and dec above.
        # This might be better done with a KD tree. Especially if cranking up to 
        # very large satellite constellations.
        distances = point_to_line_distance(
            sat_ra_1[above_illum_indx],
            sat_dec_1[above_illum_indx],
            sat_ra_2[above_illum_indx],
            sat_dec_2[above_illum_indx],
            pointing_ras_rad[above_illum_indx],
            pointing_decs_rad[above_illum_indx],
        )

        close = np.where(distances < mask_width)[0]
        
        # Could set a "close pass" value, maybe one degree, use that instead of the
        # mask width above, then for each indx in close, do a higher time-resolution
        # calculation of the satllite path and check if it gets within mask_width.
        # there is an example in the original check_pointings method on the base class
        
        # Note, could have repeat values in result as an object can be hit by more than one streak
        return input_id_indx[above_illum_indx][close]

In [3]:
ss_observations_file = 'franken_v2.99_10yrs__granvik_pha_5k_obs.txt'
ss_objects = pd.read_csv(ss_observations_file, delim_whitespace=True, comment="#")

In [4]:
ss_objects

,obj_id,time,ra,dec,dradt,ddecdt,phase,solarelon,helio_dist,geo_dist,...,night,observationStartMJD,rotSkyPos,seeingFwhmEff,seeingFwhmGeom,solarElong,visitExposureTime,dmag_color,dmag_trail,dmag_detect
0,0,60335.376576,200.306388,-7.652799,0.062432,-0.047629,18.084069,103.927280,3.078399,2.689331,...,117,60335.376576,193.188574,1.204662,1.042232,103.873779,30.0,-0.408934,0.003293,0.002018
1,0,60338.345698,200.486919,-7.786760,0.048210,-0.042903,17.916341,106.727755,3.066034,2.633845,...,120,60338.345698,158.408848,0.838235,0.741030,107.330993,30.0,-0.399899,0.004329,0.002694
2,0,60338.347056,200.486985,-7.786819,0.048189,-0.042895,17.916248,106.729052,3.066028,2.633820,...,120,60338.347056,158.208310,0.932026,0.818126,106.011239,30.0,-0.399899,0.003590,0.002210
3,0,60345.366386,200.734580,-8.044035,0.010943,-0.030443,17.319908,113.529677,3.036256,2.504955,...,127,60345.366386,106.880672,0.972575,0.851457,113.957974,30.0,-0.457036,0.000882,0.000514
4,0,60345.377677,200.734704,-8.044378,0.010860,-0.030368,17.318656,113.540886,3.036208,2.504750,...,127,60345.377677,106.880672,0.976790,0.854922,113.969424,30.0,-0.399899,0.000870,0.000507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626964,4999,62912.109752,98.026556,14.867500,-0.184410,0.000712,12.684746,131.523909,3.367264,2.630336,...,2694,62912.109752,249.912859,1.603891,1.370399,130.861848,30.0,0.389483,0.009691,0.006410
1626965,4999,62989.961830,101.022447,14.727487,0.211810,-0.019611,17.504399,58.257559,2.850319,3.248659,...,2772,62989.961830,278.875634,1.052494,0.917150,58.850837,15.0,-0.457036,0.007379,0.004768
1626966,4999,62989.964943,101.023129,14.727426,0.211868,-0.019630,17.504120,58.255246,2.850296,3.248677,...,2772,62989.964943,279.727029,1.190732,1.030782,58.847859,15.0,-0.457036,0.005947,0.003780
1626967,4999,62989.968056,101.023811,14.727365,0.211927,-0.019649,17.503841,58.252932,2.850273,3.248695,...,2772,62989.968056,280.557957,1.277623,1.102207,58.844882,15.0,-0.457036,0.005252,0.003310


In [5]:
tles = starlink_tles_v2()
print('N satellites = ', np.size(tles))
constellation = SConstellation(tles)

N satellites =  29988


In [6]:
visit_times = ss_objects['visitExposureTime']+2


In [ ]:
indices = []

# With a million object detections, we're going to need to chunk this up
breaks = np.floor(np.linspace(0,len(ss_objects)-1, 100)).astype(int)
counter = 0
for lower, upper in zip(breaks[0:-1], breaks[1:]):
    print(counter)
    # this step is the grind. Generates a few arrays that are n_detections X n_satellites, so
    # potential to gobble up a few GB of memory. Looks like I got up to 10-15 GB on a 6.8k x 30k run.
    indx = constellation.check_positions(ss_objects['ra'].values[lower:upper],
                                         ss_objects['dec'].values[lower:upper],
                                         ss_objects['observationStartMJD'].values[lower:upper],
                                         visit_times.values[lower:upper])
    indices.append(indx+lower)
    counter += 1
indx = np.concatenate(indices)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [ ]:
print(indx.size)
indx

1784


array([  10284,    9721,   10481, ..., 1619575, 1621582, 1612616])

In [ ]:
# so, we really didn't lose that many observations at all, 27 out of 6810
# let's drop those out of the dataframe
final_data = ss_objects.drop(index=indx)

In [ ]:
final_data.shape

(1625188, 27)

In [11]:
# And now we can put that back as a new txt file with the streaked objects removed
final_data.to_csv('franken_v2.99_10yrs__granvik_pha_5k_obs_streaked.txt', index=False, sep=' ')

In [12]:
breaks = np.floor(np.linspace(0,len(ss_objects), 10)).astype(int)